In [1]:
%%capture
%run RC4.ipynb
%run SDES.ipynb
%run Diffie-Hellman.ipynb

import socket
from _thread import start_new_thread

key = None
cipher = None
ciphers = {'rc4': RC4, 'sdes': SDES}
clients = []
server = None

def listen_client(client, addr):
    print(addr[0], 'connected!')
    clients.append([addr[0], client])
    while clients:
        try:
            show_msg(addr[0], client.recv(2048))
        except:
            pass

def listen_server():
    while server:
        try:
            conn, addr = server.accept()
            start_new_thread(listen_client, (conn, addr))
        except:
            pass

def stop_server():
    global key
    global cipher
    global clients
    global server
    key = None
    cipher = None
    for client in clients:
        client[1].close()
    clients = []
    if server:
        server.close()
        server = None

def start_server(ip, port):
    global server
    stop_server()
    server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
    server.bind((ip, port))
    server.listen(100)
    start_new_thread(listen_server, ())

def command_check(text):
    global key
    global cipher
    if text[:6] == '\crypt':
        cipher = ciphers.get(text[7:text.find(' ', 8)])
        if cipher:
            key = text[text.find(' ', 8)+1:]
        else:
            key = None
        return False
    return True

def show_msg(ip, text):
    text = str(text, 'utf-8')
    if command_check(text) and cipher:
        print(ip, ':', cipher(key, text))
    else:
        print(ip, ':', text)

def list_clients():
    for c in clients:
        print(c[0])

def send_msg(ip, text):
    for client in clients:
        if client[0] == ip:
            try:
                if command_check(text) and cipher:
                    client[1].send(cipher(key, text).encode())
                else:
                    client[1].send(bytes(text, 'utf-8'))
            except:
                pass

In [2]:
start_server('localhost', 5354)

127.0.0.1 connected!


In [3]:
list_clients()

127.0.0.1


In [8]:
send_msg('127.0.0.1', str(diffie_hellman_public(353,3,233)))

In [7]:
print(diffie_hellman_key(353,233,40))

160


In [5]:
stop_server()

In [ ]:
# Faltando
# no início: Trocar chave de sessão usando Diffie-Hellman, configurar q e alfa

# \crypt [sdes | rc4 | none] [key]
# tudo UTF-8

# Exemplos:
# \crypt sdes "1010101010"
# \crypt rc4 "key"
# \crypt none

# diffie_hellman_public(prime, alfa, MY_SECRET_NUMBER)
# diffie_hellman_key(prime, source_secret, destination_public_key)

# prime = 353
# alfa = 3